In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from multirtd.dynamics.dubins_model import dubins_traj, dubins_traj_new, linearize_dynamics
from multirtd.reachability.dubins_reachability import dlqr_calculate
from multirtd.reachability.zonotope import Zonotope, cov_to_zonotope
from multirtd.utils import rand_in_bounds, plot_ellipse

%load_ext autoreload
%autoreload 2

# ERS

### Monte carlo

In [ ]:
# Initial state (no uncertainty)
x0 = np.array([0, 0, 0])  # [x, y, theta]

# Monte carlo over 1000 samples
dt = 0.1
N_MC = 1000
K = 100  # Number of timesteps
sigma = 0.03  # rotational velocity sigma

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity

x_samples = []
for i in range(N_MC):
    u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
    x = dubins_traj(x0, u, dt)
    x_samples.append(x)

# Plot the trajectory
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for x in x_samples:
    ax1.plot(x[:,0], x[:,1], 'b', alpha=0.1)
    ax2.scatter(x[-1,0], x[-1,1])
plt.axis('equal')
plt.show()

### Robotics toolbox

In [ ]:
from roboticstoolbox.mobile import Bicycle, RandomPath

V = np.diag([0.2, np.radians(0.5)])
robot = Bicycle(covar=V, dt=0.1)
robot.control = RandomPath(workspace=10)

In [ ]:
N_MC = 100
x_samples = []
fig, ax = plt.subplots(figsize=(5, 5))
for i in range(N_MC):
    x = robot.run(10)
    x_samples.append(x)
    ax.plot(x[:, 0], x[:, 1])
ax.axis('equal')

### Open loop dynamics with noise

In [ ]:
def diff_drive(vl, vr, sigma, N, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    x = np.zeros((N,3))
    for i in range(N-1):
        noise = np.random.normal(0, sigma, 2)
        dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[i,2]),
                       0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[i,2]),
                       (vr - vl + noise[0] - noise[1]) / d])
        x[i+1] = x[i] + dx * dt
    return x

In [ ]:
N_MC = 1000
N = 100
d = 0.1
dt = 0.1
sigma = 0.01
vl = 0.15
vr = 0.2

x_MC = np.zeros((N_MC, N, 3))
for i in range(N_MC):
    x = diff_drive(vl, vr, sigma, N, d, dt)
    x_MC[i] = x
    plt.plot(x[:,0], x[:,1])
plt.axis('equal')
plt.show()

In [ ]:
for i in range(N_MC):
    plt.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
    #plt.plot(x_MC[i,-1,0], x_MC[i,-1,1], 'b.', markersize=1)
plt.axis('equal')
plt.show()

In [ ]:
# Nominal trajectory
v = (vl + vr) / 2
w = (vr - vl) / d
u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

plt.plot(x_nom[:, 0], x_nom[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
def check_enclosure(c, Q, pts):
    """Return fraction of points enclosed by the ellipse."""
    n = pts.shape[0]
    d = pts - c
    return np.sum(np.sum(d @ np.linalg.inv(Q) * d, axis=1) < 1) / n

In [ ]:
# Open-loop RRBT

Sigma = np.zeros((3, 3))
Q = sigma**2 * np.eye(2)
P = 0.997

fig, ax = plt.subplots(figsize=(5, 5))
for i in range(1,N):
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    Sigma = A @ Sigma @ A.T + C @ Q @ C.T
    if i % 10 == 0:
        plot_ellipse(ax, x_nom[i], Sigma[:2,:2], conf=P)
        print(check_enclosure(x_nom[i][:-1], -2*np.log(1-P)*Sigma[:2,:2], x_MC[:,i,0:2]))
        ax.scatter(x_MC[:,i,0], x_MC[:,i,1], s=1)

# for i in range(N_MC):
#     ax.plot(x_MC[i,0:N:10,0], x_MC[i,0:N:10,1], 'b.', markersize=1)
ax.axis('equal')

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.8)
plt.show()

Process uncertainty - noise in left and right wheel speeds

Measurement uncertainty - noise in ranging beacon measurements

In [ ]:
beacon_positions = np.array([[-1.0, -1.0], 
                             [5.0, -1.0], 
                             [-1.0, 5.0], 
                             [5.0, 5.0]])

def get_beacon_measurements(x, beacon_positions, sigma):
    """Return beacon measurements for a given state and beacon positions
    
    """
    z = np.zeros(beacon_positions.shape[0])
    for i in range(beacon_positions.shape[0]): 
        z[i] = np.linalg.norm(x[:2] - beacon_positions[i]) + np.random.normal(0, sigma)
    return z

def get_beacon_jacobian(x, beacon_positions):
    """Return Jacobian of beacon measurements for a given state and beacon positions
    
    """
    H = np.zeros((beacon_positions.shape[0], 3))
    for i in range(beacon_positions.shape[0]):
        H[i, :2] = (x[:2] - beacon_positions[i]) / np.linalg.norm(x[:2] - beacon_positions[i])
    return H

x = np.array([0.0, 0.0, 0.0])
sigma = 0.1
z = get_beacon_measurements(x, beacon_positions, sigma)

In [ ]:
def diff_drive_step(x, vl, vr, sigma, d, dt):
    """Diff drive model with uncertainty in the wheel velocities.

    d : distance between wheels

    """
    noise = np.random.normal(0, sigma, 2)
    dx = np.array([0.5 * (vl + vr + noise[0] + noise[1]) * np.cos(x[2]),
                    0.5 * (vl + vr + noise[0] + noise[1]) * np.sin(x[2]),
                    (vr - vl + noise[0] - noise[1]) / d])
    return x + dx * dt

In [ ]:
# Nominal trajectory
x0 = np.array([0.0, 0.0, 0.0])
dt = 0.1
d = 0.1
N = 300
vl = 0.195
vr = 0.2
v = (vl + vr) / 2
w = (vr - vl) / d
u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

plt.plot(x_nom[:, 0], x_nom[:, 1])
# Plot beacons
plt.plot(beacon_positions[:, 0], beacon_positions[:, 1], 'r*')
plt.axis('equal')
plt.show()

In [ ]:
# Test EKF
x = np.array([0.0, 0.0, 0.0])
x_est = np.array([0.0, 0.0, 0.0])
P = 0.1 * np.eye(3)
motion_sigma = 0.01  # Uncertainty in left/right wheel velocities
range_sigma = 0.1
Q_motion = motion_sigma**2 * np.eye(2)
R_sense = range_sigma**2 * np.eye(4)

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

x_traj = np.zeros((N, 3))
x_traj_est = np.zeros((N, 3))

for i in range(1, N):
    # Linearize about nominal trajectory
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    H = get_beacon_jacobian(x_nom[i], beacon_positions)

    # True dynamics
    K = dlqr_calculate(A, B, Q_lqr, R_lqr)
    u = u_nom[i] + K @ (x_nom[i] - x_est)
    vl = u[0] - u[1] * d / 2
    vr = u[0] + u[1] * d / 2
    x = diff_drive_step(x, vl, vr, motion_sigma, d, dt)

    # Measurement
    z = get_beacon_measurements(x, beacon_positions, range_sigma)
    
    # Predict step
    x_est = diff_drive_step(x_est, vl, vr, motion_sigma, d, dt)
    P = A @ P @ A.T + C @ Q_motion @ C.T
    # Update step
    # H = get_beacon_jacobian(x_est, beacon_positions)
    K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R_sense)
    x_est = x_est + K @ (z - get_beacon_measurements(x_est, beacon_positions, range_sigma))
    P = (np.eye(P.shape[0]) - K @ H) @ P

    x_traj[i] = x
    x_traj_est[i] = x_est

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(x_traj[:, 0], x_traj[:, 1], 'b')
plt.plot(x_traj_est[:, 0], x_traj_est[:, 1], 'r')
plt.plot(x_nom[:, 0], x_nom[:, 1], 'g')
plt.plot(beacon_positions[:, 0], beacon_positions[:, 1], 'r*')
plt.axis('equal')
plt.show()

In [ ]:
from tqdm import tqdm

# Monte carlo rollouts

N_MC = 300


motion_sigma = 0.01 * np.ones(N)  # Uncertainty in left/right wheel velocities
motion_sigma[100:200] = 0.05 
range_sigma = 0.1 * np.ones(N)
#range_sigma[100:200] = 1.0 

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

rollouts = np.zeros((N_MC, N, 3))

for k in tqdm(range(N_MC)):

    # Initialize
    x = np.array([0.0, 0.0, 0.0])
    x_est = np.array([0.0, 0.0, 0.0])
    P = 0.1 * np.eye(3)

    for i in range(1, N):
        Q_motion = motion_sigma[i]**2 * np.eye(2)
        R_sense = range_sigma[i]**2 * np.eye(4)
        
        # Linearize about nominal trajectory
        A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
        C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                    [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                    [-dt/d, dt/d]])
        H = get_beacon_jacobian(x_nom[i], beacon_positions)

        # True dynamics
        K = dlqr_calculate(A, B, Q_lqr, R_lqr)
        u = u_nom[i] + K @ (x_nom[i] - x_est)
        vl = u[0] - u[1] * d / 2
        vr = u[0] + u[1] * d / 2
        x = diff_drive_step(x, vl, vr, motion_sigma[i], d, dt)

        # Measurement
        z = get_beacon_measurements(x, beacon_positions, range_sigma[i])
        
        # Predict step
        x_est = diff_drive_step(x_est, vl, vr, motion_sigma[i], d, dt)
        P = A @ P @ A.T + C @ Q_motion @ C.T
        # Update step
        # H = get_beacon_jacobian(x_est, beacon_positions)
        K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R_sense)
        x_est = x_est + K @ (z - get_beacon_measurements(x_est, beacon_positions, range_sigma[i]))
        P = (np.eye(P.shape[0]) - K @ H) @ P

        rollouts[k,i] = x

In [ ]:
plt.figure(figsize=(7, 7))
for i in range(N_MC):
    plt.plot(rollouts[i,:,0], rollouts[i,:,1], 'b', alpha=0.1)
plt.plot(x_nom[:, 0], x_nom[:, 1], 'g', linewidth=2)
plt.axis('equal')
plt.show()

### RRBT Propagation

Relies on nominal trajectory - linearizes dubin's dynamics to get $(A_t, B_t)$ around nominal trajectory

Uses ranging beacon measurement setup, linearized measurement model around nominal trajectory to get $(C_t)$

Uses LQR to get $(K_t)$

In [ ]:
Sigma = 1e-4 * np.eye(3)  # state estimation covariance
Lambda = np.zeros((3, 3))  # uncertainty from not having yet taken observations

Q_lqr = np.eye(3)
R_lqr = np.eye(2)

# motion_sigma = 0.01 * np.ones(N)  # Uncertainty in left/right wheel velocities
# # motion_sigma[100:200] = 0.05 
# range_sigma = 0.1 * np.ones(N)
# range_sigma[100:200] = 1.0 

p_enc = np.zeros(N)
P = 0.9999

fig, ax = plt.subplots(figsize=(10, 10))
for i in range(1,N):
    Q_motion = motion_sigma[i]**2 * np.eye(2)
    R_sense = range_sigma[i]**2 * np.eye(4)
    
    # Linearize about nominal trajectory
    A, B = linearize_dynamics(x_nom[i], u_nom[i], dt)
    C = np.array([[0.5*dt*np.cos(x_nom[i,2]), 0.5*dt*np.cos(x_nom[i,2])], 
                  [0.5*dt*np.sin(x_nom[i,2]), 0.5*dt*np.sin(x_nom[i,2])], 
                  [-dt/d, dt/d]])
    H = get_beacon_jacobian(x_nom[i], beacon_positions)
    K = dlqr_calculate(A, B, Q_lqr, R_lqr)
    
    # Covariance prediction
    Sigma = A @ Sigma @ A.T + C @ Q_motion @ C.T
    L = Sigma @ H.T @ np.linalg.inv(H @ Sigma @ H.T + R_sense)

    # Covariance update
    Lambda = (A - B @ K) @ Lambda @ (A - B @ K).T + L @ H @ Sigma
    Sigma = Sigma - L @ H @ Sigma

    if i % 1 == 0:
        cov = Sigma[:2,:2] + Lambda[:2,:2]
        #plot_ellipse(ax, x_nom[i], cov, conf=P)
        #ax.plot(rollouts[:,i-1,0], rollouts[:,i-1,1], 'b.', markersize=1)

        Z_enc = cov_to_zonotope(cov, P=P, c=x_nom[i][:-1]) 
        Z_enc.plot(ax, color='b', alpha=0.2, line_alpha=0.2)

        # Q = -2 * np.log(1 - P) * (Sigma[:2,:2] + Lambda[:2,:2])
        # print(check_enclosure(x_nom[i][:-1], Q, rollouts[:,i-1,:-1]))

# for i in range(N_MC):
#     ax.plot(rollouts[i,0:N:10,0], rollouts[i,0:N:10,1], 'b.', markersize=1)
for i in range(N_MC):
    plt.plot(rollouts[i,:,0], rollouts[i,:,1], 'black', alpha=0.2)
ax.axis('equal')

plt.plot(x_nom[:, 0], x_nom[:, 1], 'g', linewidth=2, label='Nominal trajectory')
# plt.legend(loc='upper left')
# ax.set_xlim(3.4, 4.4)
# ax.set_ylim(3, 4)
plt.show()

Test ellipse to zonotope conversion

In [ ]:
P = 0.997
N = 100000
c = np.array([0, 0])

fig, ax = plt.subplots(figsize=(7, 7))
cov = np.array([[2, 1], [1, 2]])
plot_ellipse(ax, c, cov, conf=P)
# Sample from the distribution
x_samples = np.random.multivariate_normal(np.array([0, 0]), cov, N)
ax.plot(x_samples[:,0], x_samples[:,1], 'b.', markersize=1)
plt.show()

In [ ]:
Q = -2 * np.log(1 - P) * cov
n_inside = 0
for x in x_samples:
    if x.T @ np.linalg.inv(Q) @ x <= 1:
        n_inside += 1
n_inside

In [ ]:
check_enclosure(c, Q, x_samples)

In [ ]:
Q

In [ ]:
from scipy.linalg import sqrtm

Q = -2 * np.log(1 - P) * cov
Tinv = sqrtm(Q)
#Tinv = 2*cov

m = 5 # number of generators to use

# m evenly spaced out points on the unit circle
theta = np.linspace(0, 2*np.pi, m, endpoint=False)
G = np.array([np.cos(theta), np.sin(theta)])

# TODO: Compute L: minimum distance between the origin and the boundary of Zonotope(0, G)
# For now, hardcoded based on m=5
L = 3.0777

Z_enc = Zonotope(c[:,None], (1/L) * Tinv @ G)

In [ ]:
Tinv

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
plot_ellipse(ax, c, cov, conf=P)
Z_enc.plot(ax, color='r')

In [ ]:
from scipy.stats import chi2
from math import erf

m = 3
n = 2
eps = np.sqrt(chi2.ppf(erf(m/np.sqrt(2)), n))
eps

In [ ]:
chi2.ppf(0.95, 2)

In [ ]:
from multirtd.zonotope import Zonotope, cov_to_zonotope

fig, ax = plt.subplots(figsize=(7, 7))
c = np.zeros(2)
cov = np.eye(2)
plot_ellipse(ax, c, cov, conf=0.997)

w, v = np.linalg.eig(cov)
G = v @ np.sqrt(np.diag(w))
zono = Zonotope(c[:,None], eps * G)
zono.plot(ax, color='r')

ax.axis('equal')
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
plt.show()

# PRS

In [ ]:
# Look over space of u_nom

# Nominal trajectory
x0 = np.array([0.0, 0.0, 0.0])
dt = 0.1
N = 100
v = 0.2

for w in np.linspace(-0.2, 0.2, 100):
    u_nom = np.array([[v, w]] * N)
    x_nom = dubins_traj(x0, u_nom, dt)
    plt.plot(x_nom[:, 0], x_nom[:, 1], label=f'w={w:.2f}')

plt.axis('equal')
plt.show()

In [ ]:
# Nominal trajectory
X0 = rand_in_bounds([-0.1, 0.1, -0.1, 0.1, -0.1, 0.1], 100)
dt = 0.1
N = 100
v = 0.2
w = 0.1

for x0 in X0:
    u_nom = np.array([[v, w]] * N)
    x_nom = dubins_traj(x0, u_nom, dt)
    plt.plot(x_nom[:, 0], x_nom[:, 1], label=f'w={w:.2f}')

plt.axis('equal')
plt.show()

In [ ]:
# Nominal trajectory
x0 = np.array([0.0, 0.0, 0.0])
dt = 0.1
N = 100
v = 0.2
w = 0.1

u_nom = np.array([[v, w]] * N)
x_nom = dubins_traj(x0, u_nom, dt)

fig, ax = plt.subplots(figsize=(7, 7))

# Control set
U = Zonotope(np.zeros((2,1)), 0.01 * np.eye(2))

# Initial set
X_0 = Zonotope(x0[:,None], 0.1 * np.eye(3))  # +/- 0.1 m in x,y, +/- 0.1 rad (5.7 deg) in theta
X = X_0
# A, B = linearize_dynamics(x_nom[0], u_nom[0], dt)
for i in range(N):
    A, B = linearize_dynamics(x_nom[i], np.zeros(2), dt)
    X = A * X + B @ u_nom[i][:,None]
    X.view([0,1]).plot(ax, color='b', alpha=0.1)

ax.plot(x_nom[:, 0], x_nom[:, 1], 'g', linewidth=3)
ax.axis('equal')
plt.show()

In [ ]:
from multirtd.dubins_model import linearize_dynamics_new

In [ ]:
# Nominal trajectory
x0 = np.array([0.0, 0.0, 0.0])
dt = 0.1
N = 100

fig, ax = plt.subplots(figsize=(7, 7))

# Control set
U = Zonotope(np.array([[0.2], [0.0]]), np.array([[0.0], [0.2]]))  # v = 0.2, w = -0.2 to 0.2

# Initial set
X_0 = Zonotope(x0[:,None], 0.1 * np.eye(3))
X = X_0
A, B = linearize_dynamics_new(x0, np.zeros(2), dt)
for i in range(N):
    # A, B = linearize_dynamics(x_nom[i], np.zeros(2), dt)
    X = A * X + B * U
    X.view([0,1]).plot(ax, color='b', alpha=0.1)

v = 0.2
for w in np.linspace(-0.2, 0.2, 100):
    u_nom = np.array([[v, w]] * N)
    x_nom = dubins_traj(x0, u_nom, dt)
    ax.plot(x_nom[:, 0], x_nom[:, 1], label=f'w={w:.2f}')
ax.axis('equal')
plt.show()